# HW4

<b>Instructions:</b> Write your answers in the empty code cells below.

<b>Penalties:</b> You will incur penalties if:
<ul>
<li>Your code is wrong</li>
<li>Your code does not work on different data</li>
<li>Your code is unnecessarily slow (you use a for loop, or you use DataFrame.apply unnecessarily, etc)
</ul>

Start by downloading the file <i>cleaned_survey.csv</i> in the same folder as this .ipynb file. The file contains the results from the student survey.  

Then, execute the code below to load the data in a  dataframe <i>df</i>. 

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import window,column,desc,col,instr,expr, pow,translate,lit
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime
import pandas as pd

spark = SparkSession \
    .builder \
    .appName("Foo") \
    .config("spark.executor.memory", "1g") \
    .config("spark.driver.memory", "1g") \
    .getOrCreate()


In [3]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("delimiter", ",")\
.option("dateFormat", "MM/dd/YYYY HH:mm")\
.load("C:/Users/ramya/Desktop/Santa Clara University/Q1/Python Alan Tan/HW4/cleaned_survey-1.csv")

df.head()

Row(_c0=0, Job=0.0, Program='MSIS', ProgSkills=4, C=1, CPP=1, CS=0, Java=0, Python=1, JS=1, R=0, SQL=1, SAS=0, Excel=1, Tableau=1, Regression=1, Classification=4, Clustering=4, Bach_0to1=0, Bach_1to3=1, Bach_3to5=0, Bach_5Plus=0, Languages=6, Expert=1)

### Data Set description

The data set contains one row for every person who filled the survey. We use <b>person</b> and <b>student</b> interchangeably.

<ul>
<li><b>Job</b>: 0 for those students without a job, 0.5 for those students with a part-time job, and 1 for those students with a full-time job
<li><b>Program</b>: program of enrollment
<li><b>C-Regression</b>: indicates whether the student knows (1) or doesn't know (0) a certain programming language or topic
<li><b>Classification</b>: indicates the level of knowledge (1-5) on classification
<li><b>Clustering</b>: indicates the level of knowledge (1-5) on clustering
<li><b>Bach_0to1</b>: 1 if time elapsed from graduation is less than a year; 0 otherwise
<li><b>Bach_1to3</b>: 1 if time elapsed from graduation is between one and three years; 0 otherwise
<li><b>Bach_3to5</b>: 1 if time elapsed from graduation is between three and five years; 0 otherwise
<li><b>Bach_5Plus</b>: 1 if time elapsed from graduation is more than five years; 0 otherwise
</ul>

In [4]:
df.show(4)

+---+---+-------+----------+---+---+---+----+------+---+---+---+---+-----+-------+----------+--------------+----------+---------+---------+---------+----------+---------+------+
|_c0|Job|Program|ProgSkills|  C|CPP| CS|Java|Python| JS|  R|SQL|SAS|Excel|Tableau|Regression|Classification|Clustering|Bach_0to1|Bach_1to3|Bach_3to5|Bach_5Plus|Languages|Expert|
+---+---+-------+----------+---+---+---+----+------+---+---+---+---+-----+-------+----------+--------------+----------+---------+---------+---------+----------+---------+------+
|  0|0.0|   MSIS|         4|  1|  1|  0|   0|     1|  1|  0|  1|  0|    1|      1|         1|             4|         4|        0|        1|        0|         0|        6|     1|
|  1|0.5|   MSIS|         3|  1|  1|  0|   1|     0|  0|  0|  1|  0|    1|      0|         0|             2|         2|        0|        0|        0|         1|        4|     1|
|  2|0.0|   MSIS|         3|  0|  0|  0|   1|     1|  0|  0|  1|  0|    1|      0|         1|             3|  

### Question 1

<p>For each programming skill level, compute:</p>
<ol>
<li><b>PropPJC2</b>:the proportion (0 to 1, computed relatively to the group size) of people that know at least one among Python and Java and their Classification knowledge has to be at least 2. 
<li><b>StdevClust</b>:the standard deviation of the Clustering knowledge (function <i>std</i>)
</ol>
<p>You should return one data frame with one row for each programming skill level and two columns named <b>PropPy_or_Java_and_Class2+</b> and <b>StdevClass</b>. Don't worry about having a Hierarchical Index for the columns.</p> 

(In Camino, pick the correct **PropPy_or_Java_and_Class2+** number for student with ProgSkills = 5 )

In [33]:
import pyspark.sql.functions as func
from pyspark.sql.functions import col, when

df=df.withColumn("PropPy_or_Java_and_Class2", when((col("Classification")>=2) & ((col("Python")+col("Java"))>=1), 1.0).otherwise(0.0))

df.groupby("ProgSkills").agg(round(mean("PropPy_or_Java_and_Class2"),3),stddev("Clustering")).\
sort(col("ProgSkills")).withColumnRenamed("round(avg(PropPy_or_Java_and_Class2), 3)","mean").show()

+----------+-----+-----------------------+
|ProgSkills| mean|stddev_samp(Clustering)|
+----------+-----+-----------------------+
|         1|  0.0|     0.7867957924694432|
|         2|0.364|     0.6875516509523286|
|         3|0.621|     1.0850529486978193|
|         4|0.583|     1.0836246694508316|
|         5|  0.5|     0.7071067811865476|
+----------+-----+-----------------------+



### Question 2

For each program, how many people have a Programming Skill knowledge of less than 4? Report one row for each existing program. 

(In Camino, fill in the number of students for MBA and MSIS in a format of MBA_total/MSIS_total. For example : 10/23 )

In [36]:
import pyspark.sql.functions as func
cond=lambda x : func.sum((when(x<4,1)).otherwise(0))

In [38]:
df.groupby("Program").agg(cond(func.col("ProgSkills"))).show()

+--------------------+-------------------------------------------------+
|             Program|sum(CASE WHEN (ProgSkills < 4) THEN 1 ELSE 0 END)|
+--------------------+-------------------------------------------------+
|   Master of Finance|                                                0|
|        Business Man|                                                1|
|            Faculty!|                                                1|
|Supply Chain Mgmt...|                                                2|
|                MSIS|                                               30|
|                 MBA|                                               13|
+--------------------+-------------------------------------------------+



### Question 3

For each Job, return the rows of those people who, within their group, know the most Languages. (In Camino, pick the correct total number of people who satisfy this condition)

In [44]:
def max_l(df):
    return df.Languages.max()

In [47]:
from pyspark.sql import Window
import pyspark.sql.functions as f

w = Window.partitionBy('Job')
df.withColumn('max_L', f.max('Languages').over(w))\
    .where(f.col('Languages') == f.col('max_L'))\
    .drop('max_L')\
    .show()

+---+---+-------+----------+---+---+---+----+------+---+---+---+---+-----+-------+----------+--------------+----------+---------+---------+---------+----------+---------+------+-------------------------+
|_c0|Job|Program|ProgSkills|  C|CPP| CS|Java|Python| JS|  R|SQL|SAS|Excel|Tableau|Regression|Classification|Clustering|Bach_0to1|Bach_1to3|Bach_3to5|Bach_5Plus|Languages|Expert|PropPy_or_Java_and_Class2|
+---+---+-------+----------+---+---+---+----+------+---+---+---+---+-----+-------+----------+--------------+----------+---------+---------+---------+----------+---------+------+-------------------------+
| 46|0.0|   MSIS|         4|  1|  1|  1|   1|     1|  1|  0|  1|  0|    1|      1|         1|             2|         2|        0|        1|        0|         0|        7|     1|                      1.0|
| 45|1.0|   MSIS|         3|  1|  1|  0|   1|     1|  1|  1|  1|  0|    0|      1|         1|             2|         2|        0|        1|        0|         0|        7|     1|       

In [50]:
df.withColumn('max_L', f.max('Languages').over(w)).select("max_L","Program","Job","Languages").show(100)

+-----+--------------------+---+---------+
|max_L|             Program|Job|Languages|
+-----+--------------------+---+---------+
|    7|                MSIS|0.0|        6|
|    7|                MSIS|0.0|        3|
|    7|                MSIS|0.0|        5|
|    7|                MSIS|0.0|        4|
|    7|                MSIS|0.0|        4|
|    7|                MSIS|0.0|        3|
|    7|Supply Chain Mgmt...|0.0|        3|
|    7|                MSIS|0.0|        4|
|    7|                MSIS|0.0|        5|
|    7|                 MBA|0.0|        1|
|    7|                 MBA|0.0|        2|
|    7|                MSIS|0.0|        6|
|    7|                MSIS|0.0|        1|
|    7|                MSIS|0.0|        3|
|    7|                 MBA|0.0|        3|
|    7|                MSIS|0.0|        3|
|    7|                MSIS|0.0|        6|
|    7|   Master of Finance|0.0|        5|
|    7|                MSIS|0.0|        1|
|    7|                 MBA|0.0|        3|
|    7|    

### Question 4

Let us define the "data science experience" of a given person as the person's largest score among Regression, Classification, and Clustering. Compute the average data science experience among all MSIS students. 
(In Camino, pick the correct number)

In [55]:
from pyspark.sql.functions import greatest
df.withColumn("data science experience",greatest(col("Regression"),col("Classification"),col("Clustering"))).select(\
                    "Regression","Classification","Clustering","data science experience").show(3)

+----------+--------------+----------+-----------------------+
|Regression|Classification|Clustering|data science experience|
+----------+--------------+----------+-----------------------+
|         1|             4|         4|                      4|
|         0|             2|         2|                      2|
|         1|             3|         3|                      3|
+----------+--------------+----------+-----------------------+
only showing top 3 rows



In [59]:
df.withColumn("data science experience",greatest(col("Regression"),col("Classification"),col("Clustering"))).\
filter(col("Program")=="MSIS").agg({"data science experience":"mean"}).show(4)

+----------------------------+
|avg(data science experience)|
+----------------------------+
|                       1.875|
+----------------------------+



### Question 5

Among those with at least one year elapsed from their Bachelor's degree, find out who is the "most knowledgeable" person. The most knowledgeable person is the one who knows Classification best (in case of ties, consider whether they know C, then CPP, then CS, then Java, then SAS). Retrieve that person's Program information. (pick the right one in Camino)

In [69]:
df.filter(col("Bach_0to1")==0).orderBy(["Classification","C","CPP","CS","Java","SAS"],ascending=False).select("Program").show(1)

+--------+
| Program|
+--------+
|Faculty!|
+--------+
only showing top 1 row



In [70]:
df.filter(col("Bach_0to1")==0).orderBy(["Classification","C"],ascending=[False,True]).select("Program").show(1)

+-------+
|Program|
+-------+
|    MBA|
+-------+
only showing top 1 row

